In [2]:
import pandas as pd
import json
import os

In [ ]:
dataframe = pd.read_csv(
    "data/metadata.csv", on_bad_lines="warn", delimiter=";;"
)  # custom delimiter ';;'
# image_file_names = os.listdir("data/images")
image_file_names = os.listdir("data/retry_error")

/tmp/ipykernel_60301/2537656179.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataframe = pd.read_csv(


In [4]:
image_file_names.sort(key=lambda x: int(x.split(".")[0]))
image_file_names

['61.png',
 '67.png',
 '70.png',
 '71.png',
 '87.png',
 '88.png',
 '99.png',
 '103.png',
 '129.png',
 '135.png',
 '136.png',
 '192.png',
 '193.png',
 '202.png',
 '203.png',
 '238.png',
 '244.png',
 '250.png',
 '266.png',
 '270.png',
 '272.png',
 '375.png',
 '402.png',
 '406.png',
 '565.png',
 '569.png',
 '593.png',
 '599.png',
 '600.png',
 '604.png',
 '624.png',
 '647.png',
 '652.png',
 '659.png',
 '729.png']

Enusring all relvent data is available


In [5]:
dataframe

,file,c2t
0,0.png,Start is connected with Check needs for projec...
1,401.png,Start is connected with Check software. If Che...
2,402.png,Begin is conected with Check your PC configura...
3,3.png,Begin is conected with SUM=0 which is then con...
4,406.png,Begin is connected with calculate total travel...
...,...,...
211,818.png,Input is connected with Filtering which is the...
212,823.png,Application is connected with Login which is t...
213,848.png,Data process is connected with Degree calculat...
214,888.png,Raw Materials is connected with Melting which ...


In [6]:
len(dataframe[~dataframe["file"].isin(image_file_names)])

181

In [6]:
for file_name in image_file_names:
    if file_name not in dataframe["file"].values:
        print(file_name)

In [7]:
dataframe[dataframe["file"] == "591.png"]["c2t"].values[0]

'Start is connected with Scan Bar code which is then connected with "Reader" decrypts message and sends it to "3D object management interface" which is further connected with Check the website. Check the website is connected with The back-end server checks data and compares it with "3D objects database" which is then connected with Does data match the detail in "3D obects database". If Does data match the detail in "3D obects database" is No? then An error message is displayed on the user screen and if Does data match the detail in "3D obects database" is Yes? then The 3D object is displayed on the user screen.'

In [8]:
len(image_file_names)

35

In [ ]:
batch_1 = image_file_names[:10] # Done
batch_2 = image_file_names[10:40] # Done
batch_3 = image_file_names[40:70] # Done
batch_4 = image_file_names[70:100] # Done
batch_5 = image_file_names[100:130] # Done
batch_6 = image_file_names[130:160] # Done
batch_7 = image_file_names[160:]

In [9]:
from pydantic import BaseModel, Field
from typing import List, Literal
import dotenv
import os
import base64
from anthropic import Anthropic
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass
from pydantic_ai.models.anthropic import AnthropicModel

dotenv.load_dotenv()


True

In [ ]:
def check_for_env(key: str):
    api_key = os.getenv(key)
    if api_key is None:
        raise Exception(f"APi key {key} not available")
    return api_key


def write_json(data, file_name):
    file_name = file_name.split(".")[0]
    with open(f"outputs/{file_name}.json", "w") as f:
        json.dump(data, f, indent=4)


def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode("utf-8")
        return base64_string


anthropic_api_key = check_for_env("ANTHROPIC_API_KEY")


class Node(BaseModel):
    """Structure of a node in the diagram"""

    id: str = Field(description="Id of the Node")
    type_of_node: Literal["process", "decision", "delay", "terminator", "start"] = (
        Field(description="The type of node")
    )
    shape: Literal["task", "gateway", "start_event", "end_event", "data_store"] = Field(
        description="The shape of the node visulally"
    )
    label: str = Field(description="Label of the node")


class Edge(BaseModel):
    """Structure of a edge in the diagram"""

    source_: str = Field(description="The ID of the edge's starting node")
    source_type: Literal["process", "decision", "delay", "terminator", "start"] = Field(
        description="The type of the edge's starting node"
    )
    source_label: str = Field(description="The label of the edge's starting node")
    target: str = Field(description="The ID of edge's end node")
    target_type: Literal["process", "decision", "delay", "terminator", "start"] = Field(
        description="The type of the edge's end node"
    )
    target_label: str = Field(description="The label of the edge's end node")
    type_of_edge: Literal["dashed", "solid"] = Field(
        default="solid", description="The type of edge, visually"
    )
    relationship_value: str = Field(
        default="",
        description="The label of the relationship in the image, if present(e.g., 'yes', 'no')",
    )
    relationship_type: Literal["follows", "branches", "depends_on"] = Field(
        default="follows",
        description="Semantic type of the relationship (e.g., 'follows', 'branches', 'depends_on')",
    )


class Graph(BaseModel):
    """Structure of the graph representing the diagram"""

    nodes: List[Node] = Field(description="Nodes from the diagram")
    edges: List[Edge] = Field(description="Edges from the diagram")


class MultiModalLLMService:
    """Service to interact with Anthropic multimodal LLMs."""

    def __init__(self, model: str):
        self.client = Anthropic(api_key=anthropic_api_key)
        self.model = model

    async def perform_task(
        self,
        diagram_path: str,
        max_tokens: int = 2000,
    ):
        """Send an image and prompt to the LLM and return structured output."""

        message_list = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",  # All images are png
                            "data": get_base64_encoded_image(diagram_path),
                        },
                    },
                ],
            }
        ]
        response = self.client.messages.create(
            model=self.model,
            max_tokens=max_tokens,
            messages=message_list,
        )
        return response.content[0].text


@dataclass
class DiagramDigitizerDependencies:
    llm_service: MultiModalLLMService
    diagram_path: str


diagram_digitizer_agent = Agent(
    AnthropicModel(
        "claude-3-5-sonnet-latest",
        api_key=anthropic_api_key,
    ),
    deps_type=DiagramDigitizerDependencies,
    result_type=Graph,
    retries=8,
    system_prompt="""You are an AI model extracting structured information from diagrams.
Important Rule: Labels must be strictly preserved without any modification, truncation, rewording, or summarization.

    Each label is an identifier and must be extracted exactly as it appears in the diagram.
    Do NOT modify punctuation, or word order.
    Do NOT attempt to summarize or shorten labels.
    The extracted JSON must maintain the full original label text exactly as it appears in the diagram.

If you are unsure about a label, extract it as-is without modification rather than trying to "fix" it.
Consider their shape and translate it's purpose as it is important for the data extraction. only give the json format of the diagram. extract all other information in lowercase and in the same format as the image. But not the labels.
""",
)


@diagram_digitizer_agent.tool
async def extract_diagram_info(ctx: RunContext[DiagramDigitizerDependencies]) -> Graph:
    """Tool to extract diagram information details from the image"""
    return await ctx.deps.llm_service.perform_task(  # type: ignore
        diagram_path=ctx.deps.diagram_path,
    )


result = None

# Doing the for loops in batches
async def main():
    global result
    for index, file_name in enumerate(image_file_names):
        try:
            image_path = f"data/retry_error/{file_name}"
            deps = DiagramDigitizerDependencies(
                llm_service=MultiModalLLMService(model="claude-3-5-sonnet-20241022"),
                diagram_path=image_path,
            )
            description = dataframe[dataframe["file"] == file_name]["c2t"].values[0]
            prompt = f"""Here is the description of the image refer this for correct labels and details "{description}" """
            print(f"INDEX = {index}")
            print(f'fetching output for "{file_name}"')
            result = await diagram_digitizer_agent.run(prompt, deps=deps)
            print(f'got the output from vlm for "{file_name}"')
            data = result.data.model_dump()
            print(f'writing json for {file_name}')
            write_json(data, file_name)
        except Exception as e:
            print(f"ERROR for {file_name} : {e} {'='*10}")
            continue
    # print(result.usage)

await main()


INDEX = 0
fetching output for "61.png"
ERROR for 61.png : Exceeded maximum retries (8) for result validation =*=*=*=*=*=*=*=*=*=*
INDEX = 1
fetching output for "67.png"
got the output from vlm for "67.png"
writing json for 67.png
INDEX = 2
fetching output for "70.png"
ERROR for 70.png : Exceeded maximum retries (8) for result validation =*=*=*=*=*=*=*=*=*=*
INDEX = 3
fetching output for "71.png"
ERROR for 71.png : Exceeded maximum retries (8) for result validation =*=*=*=*=*=*=*=*=*=*
INDEX = 4
fetching output for "87.png"
ERROR for 87.png : Exceeded maximum retries (8) for result validation =*=*=*=*=*=*=*=*=*=*
INDEX = 5
fetching output for "88.png"
ERROR for 88.png : Exceeded maximum retries (8) for result validation =*=*=*=*=*=*=*=*=*=*
INDEX = 6
fetching output for "99.png"
ERROR for 99.png : Exceeded maximum retries (8) for result validation =*=*=*=*=*=*=*=*=*=*
INDEX = 7
fetching output for "103.png"
ERROR for 103.png : Exceeded maximum retries (8) for result validation =*=*=*=*=